# Metaflow with Tempo Example

We will train two models and deploy them with tempo within a Metaflow pipeline. To understand the core example see [here](https://tempo.readthedocs.io/en/latest/examples/multi-model/README.html)

![archtecture](architecture.png)

## MetaFlow Prequisites

Install metaflow

```
pip install metaflow
```

[Install Metaflow with remote AWS support](https://docs.metaflow.org/metaflow-on-aws/metaflow-on-aws).




## Tempo Requirements

For deploy to a remote Kubernetes cluster:

### GKE Cluster

Create a GKE cluster and install Seldon Core on it using [Ansible to install Seldon Core on a Kubernetes cluster](https://github.com/SeldonIO/ansible-k8s-collection).

For GKE we will need to create two files in the flow src folder:

```bash
kubeconfig.yaml
gsa-key.json
```

Follow the steps outlined in [GKE server authentication](https://cloud.google.com/kubernetes-engine/docs/how-to/api-server-authentication#environments-without-gcloud).



In [ ]:
!kubectl create ns production

In [ ]:
!kubectl create -f k8s/tempo-pipeline-rbac.yaml -n production

Create a Secret from the `k8s/s3_secret.yaml.tmpl` file by adding your AWS Key that can read from S3 and saving as `k8s/s3_secret.yaml`

In [ ]:
!kubectl create -f k8s/s3_secret.yaml -n production

## Run Flow locally to deploy to Docker

In [2]:
!python src/irisflow.py \
    --environment=conda \
    run --tempo-on-docker true

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-08 15:29:47.141 Workflow starting (run-id 31):
2021-08-08 15:29:48.643 [31/start/165 (pid 2467618)] Task is starting.
2021-08-08 15:29:52.492 [31/start/165 (pid 2467618)] Task finished successfully.
2021-08-08 15:29:54.118 [31/train_sklear

2021-08-08 14:31:00.874 [31/tempo/169 (pid 2468001)] Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
2021-08-08 14:31:00.874 [31/tempo/169 (pid 2468001)] Collecting rsa<5,>=3.1.4
2021-08-08 14:31:00.874 [31/tempo/169 (pid 2468001)] Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Collecting pyasn1-modules>=0.2.1
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Collecting cachetools<5.0,>=2.0.0
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Collecting pyasn1<0.5.0,>=0.4.6
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
2021-08-08 14:31:00.875 [31/tempo/169 (pid 2468001)] Collecting pyparsing>=2.0.2
2021-08-08 14:31:00.875 [31/tem

2021-08-08 14:31:11.392 [31/tempo/169 (pid 2468001)] Preparing transaction: ...working... done
2021-08-08 14:31:11.492 [31/tempo/169 (pid 2468001)] Verifying transaction: ...working... done
2021-08-08 14:31:11.610 [31/tempo/169 (pid 2468001)] Executing transaction: ...working... done
2021-08-08 14:31:45.907 [31/tempo/169 (pid 2468001)] Remove all packages in environment /home/clive/anaconda3/envs/tempo-38425be0-c06f-4783-bde2-95bf31bf1c5a:
2021-08-08 14:31:45.907 [31/tempo/169 (pid 2468001)] 
2021-08-08 14:31:45.907 [31/tempo/169 (pid 2468001)] Insights Manager not initialised as empty URL provided.
2021-08-08 14:31:55.955 [31/tempo/169 (pid 2468001)] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-08 15:32:00.792 [31/tempo/169 (pid 2468001)] Task finished successfully.
2021-08-08 15:32:02.670 [31/end/170 (pid 2469464)] Task is starting.
2021-08-08 15:32:06.082 [31/end/170 (pid 2469464)] Task finished successfully.
2021

Use the saved client from the Flow to make predictions

In [3]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [4]:
client = run.data.client_model

In [5]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}

## Run Flow on AWS and Deploy to Kubernetes

In [6]:
!python src/irisflow.py \
    --environment=conda \
    --with batch:image=seldonio/seldon-core-s2i-python37-ubi8:1.10.0-dev \
    run

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-08 15:33:21.881 Workflow starting (run-id 32):
2021-08-08 15:33:23.615 [32/start/172 (pid 2469999)] Task is starting.
2021-08-08 14:33:24.695 [32/start/172 (pid 2469999)] [aa141922-797f-4e1e-8216-49b77312a969] Task is starting (status SUBM

2021-08-08 14:35:52.108 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Downloading code package...
2021-08-08 14:35:52.612 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Code package downloaded.
2021-08-08 14:35:52.622 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Task is starting.
2021-08-08 14:35:52.987 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Bootstrapping environment...
2021-08-08 14:36:09.321 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Environment bootstrapped.
2021-08-08 14:36:18.563 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Task finished with exit code 0.
2021-08-08 15:36:19.404 [32/join/175 (pid 2470870)] Task finished successfully.
2021-08-08 15:36:21.203 [32/tempo/176 (pid 2471152)] Task is starting.
2021-08-08 14:36:22.364 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Task is starting (status SUBMITTED)...
2021-08-08 14:

2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Ran pip subprocess with arguments:
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] ['/opt/conda/envs/tempo-3b5a29db-9160-4897-89c8-3e5a250f8186/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.idvat6xa.requirements.txt']
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Pip subprocess output:
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting mlops-tempo
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Downloading mlops_tempo-0.3.0-py3-none-any.whl (74 kB)
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting mlserver==0.3.2
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Downloading mlserver-0.3.2-py3-none-

2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Using cached h11-0.12.0-py3-none-any.whl (54 kB)
2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting asgiref>=3.3.4
2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Using cached asgiref-3.4.1-py3-none-any.whl (25 kB)
2021-08-08 14:37:51.095 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Installing collected packages: zipp, typing-extensions, pycparser, urllib3, pyasn1, importlib-metadata, idna, charset-normalizer, cffi, starlette, six, rsa, requests, pydantic, pyasn1-modules, oauthlib, multidict, h11, cryptography, click, cachetools, asgiref, yarl, websocket-client, uvicorn, requests-oauthlib, pyyaml, python-dateutil, pyparsing, protobuf, orjson, numpy, grpcio, google-auth, fastapi, chardet, Authlib, attrs, async-timeout, seldon-deploy-sdk, redis, python-rclone, packaging, mlser

2021-08-08 14:38:03.270 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Remove all packages in environment /opt/conda/envs/tempo-3b5a29db-9160-4897-89c8-3e5a250f8186:
2021-08-08 14:38:03.270 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]
2021-08-08 14:38:08.608 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Insights Manager not initialised as empty URL provided.
2021-08-08 14:38:18.854 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-08 14:38:23.019 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Task finished with exit code 0.
2021-08-08 15:38:23.844 [32/tempo/176 (pid 2471152)] Task finished successfully.
2021-08-08 15:38:24.892 [32/end/177 (pid 2471980)] Task is starting.
2021-08-08 14:38:25.925 [32/end/177 (pid 2471980)] [d6392d28-0aec-416f-ac0e-670e0ef63c0b] Task is st

Use the saved client from the Flow to make predictions

In [7]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [8]:
client = run.data.client_model

In [9]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}